In [33]:
import pandas as pd
import numpy as np
from network import Network

In [34]:
recent_matches = pd.read_csv('data/recent_seasons_unnormalized.csv')
pca_matches = pd.read_csv('data/recent_seasons_unnormalized.csv')
labels = pd.read_csv('data/labels_recent_seasons.csv')

recent_matches = recent_matches.replace(-1, np.nan)
labels = labels.replace(-1, np.nan)

In [35]:
home_odds_labels = ['B365H', 'BWH', 'IWH', 'LBH', 'PSH', 'WHH', 'SJH', 'VCH', 'GBH', 'BSH']
draw_odds_labels = ['B365D', 'BWD', 'IWD', 'LBD', 'PSD', 'WHD', 'SJD', 'VCD', 'GBD', 'BSD']
away_odds_labels = ['B365A', 'BWA', 'IWA', 'LBA', 'PSA', 'WHA', 'SJA', 'VCA', 'GBA', 'BSA']
all_odds_labels = [home_odds_labels, draw_odds_labels, away_odds_labels]
combined_odds_labels = home_odds_labels + draw_odds_labels + away_odds_labels

matches = recent_matches[combined_odds_labels]

In [36]:
max_odds = pd.DataFrame(columns=['home_odds', 'draw_odds', 'away_odds'])
for i, row in matches.iterrows():
    max_odds.loc[i, 'home_odds'] = np.max(matches.loc[i, home_odds_labels])
    max_odds.loc[i, 'draw_odds'] = np.max(matches.loc[i, draw_odds_labels])
    max_odds.loc[i, 'away_odds'] = np.max(matches.loc[i, away_odds_labels])
    
# max_odds.to_csv('data/recent_seasons_max_odds.csv')

In [37]:
# predictions = pd.DataFrame(columns=['home_prob', 'draw_prob', 'away_prob'])

# for i in range(len(matches)):
#     vals = np.array([np.random.rand() for i in range(3)])
#     vals /= np.sum(vals)
#     predictions.loc[i, 'home_prob'] = vals[0]
#     predictions.loc[i, 'draw_prob'] = vals[1]
#     predictions.loc[i, 'away_prob'] = vals[2]

In [38]:
BetNet = Network(pca_matches.shape[1])
BetNet.load_weights("weights-improvement-400-0.48.hdf5")  # Most recent weights

predictions = pd.DataFrame(columns=['home_prob', 'draw_prob', 'away_prob'])

for r, match in pca_matches.iterrows():
    net_pred = BetNet.model.predict(np.array([match.values]))[0]
    predictions.loc[r, 'home_prob'] = net_pred[0]
    predictions.loc[r, 'draw_prob'] = net_pred[1]
    predictions.loc[r, 'away_prob'] = net_pred[2]

BUILDING NETWORK...
Data Dimension:  74
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 512)               38400     
_________________________________________________________________
activation_21 (Activation)   (None, 512)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 128)               65664     
_________________________________________________________________
activation_22 (Activation)   (None, 128)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 128)               16512     
_________________________________________________________________
activation_23 (Activation)   (None, 128)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 

In [39]:
home_, draw_, away_ = (0, 1, 2)

def get_result_corresponding_to_betting_org(org):
    for result, odds_orgs in enumerate(all_odds_labels):
        if org in odds_orgs:
            return result


def get_bet_org_highest_exp(odds, predictions):
    best_exp, best_org, best_odds, chosen_result = 0, None, 0, None
    for org, odds_ in odds.iteritems():
        result = get_result_corresponding_to_betting_org(org)
        exp = odds_ * predictions[result]
        if exp > best_exp:
            best_exp, best_org, best_odds, chosen_result = exp, org, odds_, result
            
    return chosen_result, best_org


def get_bet_org_highest_prob(odds, predictions):
    chosen_result = int(np.argmax(predictions.values))
    best_org = pd.Series.argmax(odds[all_odds_labels[chosen_result]])
    # print(best_org, ['home', 'draw', 'away'][chosen_result], odds[best_org])
    return chosen_result, best_org
    
    
def get_bet(odds, predictions, funds):
    bet_side, bet_org = get_bet_org_highest_prob(odds, predictions) # get_bet_org_highest_exp(odds, predictions)
    # bet_amount = np.sqrt(funds) * odds[bet_org] * predictions[bet_side]
    # bet_amount = min(funds/10, bet_amount)
    bet_amount = min(funds / 10, 100000 / odds[bet_org])
                      
    return bet_amount, bet_side, bet_org


def get_profit(bet_amount, bet_side, odds, labels_row):
    if labels_row[bet_side]:
        return bet_amount * odds - bet_amount
    else:
        return -bet_amount

In [40]:
funds = 50
initial_funds = funds

n_correct = 0

funds_list = [50]

for r in range(320, 370):#509):  # 2017 / 2018 season
    bet_amount, bet_side, bet_org = get_bet(matches.loc[r], predictions.loc[r], funds)
    odds = matches.loc[r, bet_org]
    profit = get_profit(bet_amount, bet_side, odds, labels.loc[r])
#     print(funds, bet_amount, profit)
    funds += profit
    funds_list.append(funds)
    
    if funds < 1:
        break

    if profit > 0:
        n_correct += 1
        
print(funds_list)
    
profit = funds - initial_funds
print('initial funds: ${:.2f}'.format(initial_funds))
print('final funds: ${:.2f}'.format(funds))
print('profit: ${0:.2f}'.format(profit))
print('profit percent: {0:.2f}%'.format(profit * 100 / initial_funds))
print('predicted {} of {} matches correctly'.format(n_correct, r))
print('prediction percent: {0:.2f}%'.format(n_correct * 100 / r))
print('lasted {} of {} matches before dropping below $1'.format(r, len(matches)))
print('made it through {0:.2f}% of matches before dropping below 1 dollar'.format(100 * r / len(matches)))

[50, 62.5, 56.25, 59.45625, 53.510625, 48.1595625, 43.34360625, 39.009245625, 39.984476765625, 43.58307967453125, 39.224771707078126, 46.0891067558168, 41.48019608023512, 37.33217647221161, 33.59895882499045, 30.2390629424914, 32.50699266317825, 35.757691929496076, 32.18192273654647, 28.96373046289182, 26.06735741660264, 27.631398861598797, 28.18402683883077, 25.365624154947696, 22.829061739452925, 20.546155565507632, 18.491540008956868, 22.467221110882594, 20.220498999794334, 27.90428861971618, 25.113859757744564, 26.495122044420516, 27.819878146641543, 28.932673272507206, 29.42452871813983, 26.482075846325845, 23.83386826169326, 21.450481435523937, 26.770200831533874, 31.722687985367642, 28.55041918683088, 25.69537726814779, 27.160013772432215, 24.444012395188995, 21.999611155670095, 25.40955088479896, 22.868595796319063, 20.58173621668716, 20.84929878750409, 21.245435464466667, 27.364120878233066]
initial funds: $50.00
final funds: $27.36
profit: $-22.64
profit percent: -45.27%
pred